In [ ]:
# 点群可視化アニメーション
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from ipyfilechooser import FileChooser
from ipywidgets import Button, VBox, HBox, Output
from IPython.display import display, HTML

# ファイル選択ウィジェット
chooser_csv = FileChooser('csv/')
button = Button(description='Load and Animate')
output = Output()

def on_button_click(button):
    with output:
        output.clear_output()
        csv_file_path = chooser_csv.selected
        if not csv_file_path:
            print("ファイルを選択してください。")
            return

        data_csv = pd.read_csv(csv_file_path)

        # カラム名の取得
        columns = data_csv.columns
        attributes = set(col.rsplit('_', 1)[0] for col in columns if col != 'frame_number')
        def get_points(data):
            points = []
            for column in attributes:
                x_col = f'{column}_x'
                y_col = f'{column}_y'
                points.append((x_col, y_col))
            return points
        
        points_csv = get_points(data_csv)
        fig, ax = plt.subplots(figsize=(16, 6))
        scatters_csv = [ax.scatter([], [], s=10) for _ in points_csv]

        def init():
            ax.set_xlim(data_csv[[x for x, y in points_csv]].min().min(), data_csv[[x for x, y in points_csv]].max().max())
            ax.set_ylim(data_csv[[y for x, y in points_csv]].max().max(), data_csv[[y for x, y in points_csv]].min().min())
            return scatters_csv

        def update(frame):
            for scatter, (x_col, y_col) in zip(scatters_csv, points_csv):
                scatter.set_offsets([[data_csv[x_col][frame], data_csv[y_col][frame]]])
            return scatters_csv

        ani = animation.FuncAnimation(fig, update, frames=len(data_csv), init_func=init, blit=True, interval=100)
        # ani.save('animation.gif', writer='pillow', fps=30)
        display(HTML(ani.to_jshtml()))


In [ ]:
# ウィジェット表示
button.on_click(on_button_click)
display(VBox([HBox([chooser_csv, button]), output]))